In [13]:
# Standard library imports
import os
import time
import warnings
import random
import glob
import pickle
from pathlib import Path
from typing import Tuple, Dict, List
import numpy as np
import pandas as pd
import requests
import zipfile
import cv2
from PIL import Image
from skimage import io, transform
from skimage.feature import hog
from skimage import data, exposure
from skimage.color import rgb2gray
from skimage.transform import resize
from collections import Counter

# ML & utilities 
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import classification_report, accuracy_score

# Data visualization
import matplotlib.pyplot as plt
plt.ion()  

In [14]:
def load_pickle_image(file_path):
    with open(file_path, "rb") as f:         
        return pickle.load(f)                 

file_paths = []                              
for vol in range(1, 9):                      
    folder = f"archive/vol{vol:02d}"         
    file_paths += sorted(glob.glob(f"{folder}/*.pck"))  

print(f"Found {len(file_paths)} total image files")

Found 736 total image files


In [15]:
# Associate labels from metadata.csv to image file paths

meta = pd.read_csv("metadata.csv")
# Map filename -> numeric label
fn_to_label = dict(zip(meta['volumeFilename'], meta['aclDiagnosis']))
label_names = {0: 'Normal', 1: 'Torn', 2: 'Partially torn'}

# Build mapping list (file_path, numeric_label, label_name_or_None)
mapped = []
for p in file_paths:
    fname = os.path.basename(p)
    label = fn_to_label.get(fname, None)
    mapped.append((p, label, label_names.get(label) if label is not None else None))


# Summary
print("Label distribution (numeric):", Counter(l for _p, l, _n in mapped if l is not None))
print("Label distribution (names):", Counter(n for _p, l, n in mapped if n is not None))

Label distribution (numeric): Counter({0: 547, 1: 144, 2: 45})
Label distribution (names): Counter({'Normal': 547, 'Torn': 144, 'Partially torn': 45})


In [ ]:
MAX_SAMPLES = 300
IMG_SIZE = (64, 64)
hog_params = dict(orientations=9, pixels_per_cell=(8,8), cells_per_block=(2,2))
AUG_PER_SAMPLE = 1

# Build per-label lists
paths_by_label = {}
for file_path, label, _name in mapped:
    if label is None:
        continue
    paths_by_label.setdefault(label, []).append(file_path)

class_labels = sorted(paths_by_label.keys())
num_classes = len(class_labels)
samples_per_class_target = max(1, MAX_SAMPLES // num_classes)

# Helper augmentation (works on resized 2D slices)
from skimage.transform import rotate as _rotate
def augment_slice(img):
    a = img.copy()
    if random.random() < 0.5:
        a = np.fliplr(a)
    ang = random.uniform(-10, 10)
    a = _rotate(a, ang, mode='edge', preserve_range=True)
    return a

hog_features_list = []
image_tensors_list = []
labels_list = []
failed_files = []
t_start = time.time()
for label in class_labels:
    file_paths_for_label = paths_by_label[label]
    if len(file_paths_for_label) == 0:
        continue
    # Sample with replacement until we reach the per-class target
    while sum(1 for lab in labels_list if lab == label) < samples_per_class_target:
        chosen_path = random.choice(file_paths_for_label)
        volume = load_pickle_image(chosen_path)
        volume_arr = np.asarray(volume)
        mid_slice = volume_arr[volume_arr.shape[0] // 2] if volume_arr.ndim == 3 else volume_arr
        if mid_slice.ndim == 3:
            mid_slice = rgb2gray(mid_slice)
        slice_resized = resize(mid_slice, IMG_SIZE, anti_aliasing=True)
        # compute HOG descriptor for this slice
        hog_descriptor = hog((slice_resized * 255).astype(np.uint8), orientations=hog_params['orientations'], pixels_per_cell=hog_params['pixels_per_cell'], cells_per_block=hog_params['cells_per_block'], visualize=False, feature_vector=True)
        hog_features_list.append(hog_descriptor)
        image_tensors_list.append(np.expand_dims(slice_resized.astype(np.float32), -1))
        labels_list.append(label)
        # perform light augmentations if we still need more examples for this class
        if sum(1 for lab in labels_list if lab == label) < samples_per_class_target:
            for _ in range(AUG_PER_SAMPLE):
                aug_slice = augment_slice(slice_resized)
                hog_descriptor_aug = hog((aug_slice * 255).astype(np.uint8), orientations=hog_params['orientations'], pixels_per_cell=hog_params['pixels_per_cell'], cells_per_block=hog_params['cells_per_block'], visualize=False, feature_vector=True)
                hog_features_list.append(hog_descriptor_aug)
                image_tensors_list.append(np.expand_dims(aug_slice.astype(np.float32), -1))
                labels_list.append(label)
t_end = time.time()

# Truncate or pad to MAX_SAMPLES
if len(hog_features_list) > MAX_SAMPLES:
    keep_idx = random.sample(range(len(hog_features_list)), MAX_SAMPLES)
    hog_features_list = [hog_features_list[i] for i in keep_idx]
    image_tensors_list = [image_tensors_list[i] for i in keep_idx]
    labels_list = [labels_list[i] for i in keep_idx]

X_hog = np.vstack(hog_features_list)
y_labels = np.array(labels_list)
X_images = np.stack(image_tensors_list)
y_image_labels = np.array(labels_list)

# Compute class weights for CNN
from sklearn.utils import class_weight
unique_classes = np.unique(y_image_labels)
cw = class_weight.compute_class_weight('balanced', classes=unique_classes, y=y_image_labels)
class_weight_map = {int(c): float(w) for c, w in zip(unique_classes, cw)}
print('Class weights:', class_weight_map)

# Train/test split for HOG features
X_hog_train, X_hog_test, y_hog_train, y_hog_test = train_test_split(X_hog, y_labels, test_size=0.2, stratify=y_labels, random_state=42)

epochs = 10
sgd_svm = SGDClassifier(loss="hinge", class_weight=class_weight_map, random_state=42)
for epoch in range(epochs):
    sgd_svm.partial_fit(X_hog_train, y_hog_train, classes=np.unique(y_hog_train))

sgd_logr = SGDClassifier(loss="log_loss", class_weight=class_weight_map, random_state=42)
for epoch in range(epochs):
    sgd_logr.partial_fit(X_hog_train, y_hog_train, classes=np.unique(y_hog_train))

print("SGD-SVM acc:", accuracy_score(y_hog_test, sgd_svm.predict(X_hog_test)))
print("SGD-LogReg acc:", accuracy_score(y_hog_test, sgd_logr.predict(X_hog_test)))

# CNN training with class weights and small augmentations baked in
X_images_train, X_images_test, y_images_train, y_images_test = train_test_split(X_images, y_image_labels, test_size=0.2, stratify=y_image_labels, random_state=42)
n_classes = len(np.unique(y_image_labels))
model = models.Sequential([
    layers.Input(shape=X_images_train.shape[1:]),
    layers.Conv2D(16, 3, activation='relu'),
    layers.MaxPool2D(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPool2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_images_train, y_images_train, epochs=50, batch_size=16, validation_data=(X_images_test, y_images_test), class_weight=class_weight_map, verbose=2)
loss, acc = model.evaluate(X_images_test, y_images_test, verbose=0)
print('\nCNN test acc:', acc)

Class weights: {0: 1.0, 1: 1.0, 2: 1.0}
SGD-SVM acc: 0.5333333333333333
SGD-LogReg acc: 0.5
Epoch 1/50


/Users/neel/Documents/GitHub/ACLInjuryDetection/venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/neel/Documents/GitHub/ACLInjuryDetection/venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/neel/Documents/GitHub/ACLInjuryDetection/venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/neel/Documents/GitHub/ACLInjuryDetection/venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/neel/Documents/GitHub/ACLInjuryDetection/venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/neel/Documents/GitHub/ACLInjuryDetection/venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: Runtim

15/15 - 1s - 45ms/step - accuracy: 0.2875 - loss: 1.0994 - val_accuracy: 0.3333 - val_loss: 1.0986
Epoch 2/50
15/15 - 0s - 6ms/step - accuracy: 0.3333 - loss: 1.0987 - val_accuracy: 0.3333 - val_loss: 1.0986
Epoch 3/50
15/15 - 0s - 6ms/step - accuracy: 0.2875 - loss: 1.0988 - val_accuracy: 0.3500 - val_loss: 1.0986
Epoch 4/50
15/15 - 0s - 6ms/step - accuracy: 0.3333 - loss: 1.0986 - val_accuracy: 0.3333 - val_loss: 1.0986
Epoch 5/50
15/15 - 0s - 6ms/step - accuracy: 0.3333 - loss: 1.0987 - val_accuracy: 0.3333 - val_loss: 1.0986
Epoch 6/50
15/15 - 0s - 6ms/step - accuracy: 0.2708 - loss: 1.0988 - val_accuracy: 0.3167 - val_loss: 1.0986
Epoch 7/50
15/15 - 0s - 6ms/step - accuracy: 0.3167 - loss: 1.0988 - val_accuracy: 0.4333 - val_loss: 1.0984
Epoch 8/50
15/15 - 0s - 6ms/step - accuracy: 0.3750 - loss: 1.0984 - val_accuracy: 0.3000 - val_loss: 1.0984
Epoch 9/50
15/15 - 0s - 6ms/step - accuracy: 0.3875 - loss: 1.0974 - val_accuracy: 0.3667 - val_loss: 1.0987
Epoch 10/50
15/15 - 0s - 6ms/